In [1]:
import time
import os
import re
import numpy as np
from six.moves import xrange
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import load_dataset2

In [2]:
data_dir = "./"
train_dir='training_inputs/'
eye = "L"

In [3]:
train_pose_dirs = np.asarray([d for d in os.listdir(os.path.join(data_dir,train_dir)) if os.path.isdir(os.path.join(data_dir,train_dir, d))])
print("[T] pose dirs", train_pose_dirs)

train_dicts = {}
for p in train_pose_dirs:
    print('pose', p)
    train_dirs = np.asarray([d for d in os.listdir(os.path.join(data_dir,train_dir, p)) if os.path.isdir(os.path.join(data_dir,train_dir, p, d))])
    print("[T] dirs", train_dirs)
 
    # load training inputs
    train_dict = load_dataset2.load(data_dir=os.path.join(data_dir,train_dir, p), dirs = train_dirs, eye = eye)
 
    train_dicts[p] = train_dict

[T] pose dirs ['0']
pose 0
[T] dirs ['0000' '0001']
./training_inputs/0\0000
./training_inputs/0\0001


In [4]:
train_dicts.keys()

dict_keys(['0'])

In [5]:
ori_imgs = []
eye_imgs = []
lid_imgs = []
for pose in train_dicts.keys():
    for uid in range(len(train_dicts[pose]['imgs_ori'])):
        for img in range(len(train_dicts[pose]['imgs_ori'][uid])):
            ori_imgs.append(train_dicts[pose]['imgs_ori'][uid][img])
            temp = train_dicts[pose]['imgs_ori'][uid][img]*np.tile(np.expand_dims(train_dicts[pose]['msk_eye'][uid][img], axis = 2), (1,1,3))
            eye_imgs.append(temp)
            temp = train_dicts[pose]['imgs_ori'][uid][img]*np.tile(np.expand_dims(train_dicts[pose]['msk_lid'][uid][img], axis = 2), (1,1,3))
            lid_imgs.append(temp)
            
ori_imgs = np.asarray(ori_imgs)
eye_imgs = np.asarray(eye_imgs)
lid_imgs = np.asarray(lid_imgs)
ori_imgs = np.expand_dims(ori_imgs,axis=1)
eye_imgs = np.expand_dims(eye_imgs,axis=1)
lid_imgs = np.expand_dims(lid_imgs,axis=1)
imgs = np.concatenate((ori_imgs,eye_imgs,lid_imgs), axis = 1)

In [6]:
imgs.shape

(18, 3, 48, 64, 3)

In [7]:
def grid_plot(samples, filepath):
    sample_shp = samples.shape
    fig = plt.figure(figsize=(sample_shp[1], sample_shp[0]))
    gs = gridspec.GridSpec(sample_shp[0], sample_shp[1])
    gs.update(wspace=0.05, hspace=0.05)
    samples = samples.reshape(-1, sample_shp[2], sample_shp[3], sample_shp[4])
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(sample_shp[2], sample_shp[3], sample_shp[4]))
    plt.savefig("./eye_imgs/"+str(filepath)+ ".png")
    plt.close("all")

In [8]:
for i in range(0,ori_imgs.shape[0],30):
    temp = imgs[i:i+30]
    grid_plot(temp,i)